# 📊 Autocorrelación (ACF) y Autocorrelación Parcial (PACF)

**Curso de Introducción a Series Temporales para Ciencia de Datos**

En este notebook aprenderemos:
- Qué es la autocorrelación y cómo interpretarla
- Qué es la autocorrelación parcial y en qué se diferencia
- Cómo visualizar ACF y PACF
- Cómo usar estos conceptos para seleccionar modelos

---

## 1. Instalación de librerías necesarias

In [ ]:
# Instalamos statsmodels si no está disponible
try:
    import statsmodels
except ImportError:
    !pip install statsmodels

# Importamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

---

## 2. ¿Qué es la Autocorrelación (ACF)?

La **autocorrelación** mide la correlación de una serie temporal consigo misma en diferentes retardos (lags).

**Pregunta clave:** ¿Qué tan relacionado está el valor de hoy con el de ayer, anteayer, hace 7 días, etc.?

### 2.1 Ejemplo simple: Serie con tendencia

In [ ]:
# Creamos una serie temporal simple con tendencia
np.random.seed(42)
tiempo = np.arange(100)
serie_tendencia = 2 * tiempo + np.random.normal(0, 10, 100)

# Visualizamos
plt.figure(figsize=(12, 4))
plt.plot(serie_tendencia, linewidth=2)
plt.title('Serie Temporal con Tendencia', fontsize=14, fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True, alpha=0.3)
plt.show()

print("📌 Observación: Esta serie tiene una tendencia ascendente clara.")
print("   Los valores consecutivos están muy relacionados entre sí.")

### 2.2 Gráfico de Autocorrelación (ACF)

In [ ]:
# Graficamos la ACF
fig, ax = plt.subplots(figsize=(12, 5))
plot_acf(serie_tendencia, lags=30, ax=ax)
plt.title('Autocorrelación (ACF) - Serie con Tendencia', fontsize=14, fontweight='bold')
plt.xlabel('Lag (Rezago)')
plt.ylabel('Autocorrelación')
plt.tight_layout()
plt.show()

print("\n🔍 Interpretación del gráfico ACF:")
print("   - Las barras azules muestran la correlación en cada lag")
print("   - El área sombreada indica el nivel de significancia estadística")
print("   - Si una barra sale del área sombreada, esa correlación es significativa")
print("   - Aquí vemos que la correlación decae MUY LENTAMENTE → señal de tendencia/no estacionariedad")

### 2.3 Calculando valores numéricos de ACF

In [ ]:
# Calculamos los primeros 10 valores de ACF
acf_values = acf(serie_tendencia, nlags=10)

print("\n📊 Valores de Autocorrelación para los primeros 10 lags:")
print("-" * 50)
for i, valor in enumerate(acf_values):
    print(f"Lag {i}: {valor:.4f}")

print("\n💡 Nota: El lag 0 siempre es 1.0 (correlación perfecta consigo mismo)")

---

## 3. ¿Qué es la Autocorrelación Parcial (PACF)?

La **autocorrelación parcial** mide la correlación entre la serie y un lag específico, **eliminando el efecto de los lags intermedios**.

**Analogía:** Imagina que estudias cómo tu estado de ánimo de hoy se relaciona con el de hace 3 días:
- **ACF:** Incluye el efecto indirecto (hoy → ayer → anteayer → hace 3 días)
- **PACF:** Solo la relación directa entre hoy y hace 3 días

### 3.1 Comparación ACF vs PACF

In [ ]:
# Creamos una serie autoregresiva AR(2)
# Esto significa: valor_actual = 0.7 * valor_anterior + 0.2 * valor_hace_2 + ruido
np.random.seed(123)
n = 200
serie_ar2 = [0, 0]

for i in range(2, n):
    nuevo_valor = 0.7 * serie_ar2[i-1] + 0.2 * serie_ar2[i-2] + np.random.normal(0, 1)
    serie_ar2.append(nuevo_valor)

serie_ar2 = np.array(serie_ar2)

# Visualizamos la serie
plt.figure(figsize=(12, 4))
plt.plot(serie_ar2, linewidth=1.5, alpha=0.8)
plt.title('Serie Temporal AR(2)', fontsize=14, fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True, alpha=0.3)
plt.show()

print("📌 Esta es una serie AR(2): cada valor depende de los 2 valores anteriores")

### 3.2 Gráficos ACF y PACF lado a lado

In [ ]:
# Creamos gráficos comparativos
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(serie_ar2, lags=20, ax=axes[0])
axes[0].set_title('Autocorrelación (ACF)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')

# PACF
plot_pacf(serie_ar2, lags=20, ax=axes[1], method='ywm')
axes[1].set_title('Autocorrelación Parcial (PACF)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('PACF')

plt.tight_layout()
plt.show()

print("\n🔍 Interpretación para serie AR(2):")
print("   ACF:  Decae gradualmente (patrón exponencial/sinusoidal)")
print("   PACF: Corta abruptamente después del lag 2")
print("   ✅ Esto confirma que es un proceso AR(2)")

---

## 4. Casos prácticos: Identificando patrones

### 4.1 Serie con Estacionalidad

In [ ]:
# Creamos una serie con patrón estacional (cada 12 períodos)
np.random.seed(456)
t = np.arange(120)
estacionalidad = 10 * np.sin(2 * np.pi * t / 12)
ruido = np.random.normal(0, 2, 120)
serie_estacional = estacionalidad + ruido

# Visualizamos
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Serie temporal
axes[0].plot(serie_estacional, linewidth=2)
axes[0].set_title('Serie con Estacionalidad (período = 12)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Tiempo')
axes[0].set_ylabel('Valor')
axes[0].grid(True, alpha=0.3)

# ACF
plot_acf(serie_estacional, lags=40, ax=axes[1])
axes[1].set_title('ACF - Observa los picos cada 12 lags', fontsize=13, fontweight='bold')

# PACF
plot_pacf(serie_estacional, lags=40, ax=axes[2], method='ywm')
axes[2].set_title('PACF', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("🔍 Interpretación:")
print("   - La ACF muestra picos significativos cada 12 lags")
print("   - Esto indica estacionalidad con período 12")
print("   - Para modelar esto, necesitarías componentes estacionales (SARIMA)")

### 4.2 Serie Ruido Blanco (sin correlación)

In [ ]:
# Ruido blanco: valores completamente aleatorios, sin correlación
np.random.seed(789)
ruido_blanco = np.random.normal(0, 1, 200)

# Visualizamos
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Serie temporal
axes[0].plot(ruido_blanco, linewidth=1, alpha=0.7)
axes[0].set_title('Ruido Blanco (valores aleatorios independientes)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Tiempo')
axes[0].set_ylabel('Valor')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# ACF
plot_acf(ruido_blanco, lags=30, ax=axes[1])
axes[1].set_title('ACF - No hay correlaciones significativas', fontsize=13, fontweight='bold')

# PACF
plot_pacf(ruido_blanco, lags=30, ax=axes[2], method='ywm')
axes[2].set_title('PACF - Tampoco hay correlaciones parciales', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("🔍 Interpretación:")
print("   - Casi todas las barras están dentro del área sombreada")
print("   - No hay autocorrelación significativa → es ruido puro")
print("   - Este es el resultado ideal para los RESIDUOS de un buen modelo")

---

## 5. Guía práctica: ¿Qué modelo usar?

### Reglas de decisión basadas en ACF y PACF

In [ ]:
# Tabla resumen
print("=" * 70)
print("GUÍA RÁPIDA: Selección de Modelos según ACF y PACF")
print("=" * 70)
print("\n1️⃣ PROCESO AR(p) - Autoregresivo")
print("   ACF:  Decae gradualmente (exponencial o sinusoidal)")
print("   PACF: Corta después del lag p")
print("   → Modelo: AR(p)")

print("\n2️⃣ PROCESO MA(q) - Media Móvil")
print("   ACF:  Corta después del lag q")
print("   PACF: Decae gradualmente")
print("   → Modelo: MA(q)")

print("\n3️⃣ PROCESO ARMA(p,q)")
print("   ACF:  Decae gradualmente")
print("   PACF: Decae gradualmente")
print("   → Modelo: ARMA(p,q)")

print("\n4️⃣ NO ESTACIONARIA")
print("   ACF:  Decae MUY lentamente o no decae")
print("   → Necesitas diferenciar la serie antes de modelar (ARIMA)")

print("\n5️⃣ ESTACIONALIDAD")
print("   ACF:  Picos significativos en múltiplos del período estacional")
print("   → Usa componentes estacionales (SARIMA)")

print("\n6️⃣ RUIDO BLANCO")
print("   ACF y PACF: Sin correlaciones significativas")
print("   → No necesitas modelar (o tu modelo ya capturó todo)")
print("=" * 70)

---

## 6. Ejemplo aplicado: Identificando el modelo correcto

Vamos a crear tres series diferentes y practicar la identificación:

In [ ]:
np.random.seed(2024)

# Serie 1: AR(1)
ar1 = [0]
for i in range(150):
    ar1.append(0.8 * ar1[-1] + np.random.normal(0, 1))
ar1 = np.array(ar1[1:])

# Serie 2: MA(1)
errores = np.random.normal(0, 1, 151)
ma1 = [errores[0]]
for i in range(1, 150):
    ma1.append(errores[i] + 0.6 * errores[i-1])
ma1 = np.array(ma1)

# Serie 3: ARMA(1,1)
arma = [0]
errores_arma = np.random.normal(0, 1, 151)
for i in range(1, 150):
    arma.append(0.7 * arma[-1] + errores_arma[i] + 0.4 * errores_arma[i-1])
arma = np.array(arma[1:])

# Función para analizar cada serie
def analizar_serie(serie, titulo):
    fig, axes = plt.subplots(3, 1, figsize=(12, 9))

    # Serie
    axes[0].plot(serie, linewidth=1.5)
    axes[0].set_title(f'{titulo} - Serie Temporal', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)

    # ACF
    plot_acf(serie, lags=20, ax=axes[1])
    axes[1].set_title('ACF', fontsize=11, fontweight='bold')

    # PACF
    plot_pacf(serie, lags=20, ax=axes[2], method='ywm')
    axes[2].set_title('PACF', fontsize=11, fontweight='bold')

    plt.tight_layout()
    plt.show()

# Analizamos cada serie
print("🔍 EJERCICIO: Identifica el tipo de proceso en cada caso\n")

print("=" * 60)
print("SERIE 1")
print("=" * 60)
analizar_serie(ar1, "Serie 1")
print("¿Qué modelo sugieres? (Piensa antes de ver la respuesta abajo)\n")

print("=" * 60)
print("SERIE 2")
print("=" * 60)
analizar_serie(ma1, "Serie 2")
print("¿Qué modelo sugieres?\n")

print("=" * 60)
print("SERIE 3")
print("=" * 60)
analizar_serie(arma, "Serie 3")
print("¿Qué modelo sugieres?\n")

### Respuestas:

In [ ]:
print("\n" + "=" * 60)
print("✅ RESPUESTAS")
print("=" * 60)
print("\nSERIE 1:")
print("  ACF → Decae gradualmente")
print("  PACF → Corta después del lag 1")
print("  ➡️ Modelo sugerido: AR(1)")

print("\nSERIE 2:")
print("  ACF → Corta después del lag 1")
print("  PACF → Decae gradualmente")
print("  ➡️ Modelo sugerido: MA(1)")

print("\nSERIE 3:")
print("  ACF → Decae gradualmente")
print("  PACF → Decae gradualmente")
print("  ➡️ Modelo sugerido: ARMA(1,1) o similar")

---

## 7. Resumen y Conclusiones

### Conceptos clave:

1. **Autocorrelación (ACF)**: Mide la correlación total entre valores separados por k períodos
   
2. **Autocorrelación Parcial (PACF)**: Mide la correlación directa, eliminando efectos intermedios

3. **Uso principal**: Identificar el orden de modelos ARIMA(p,d,q)

### Checklist para análisis de series temporales:

Visualiza tu serie temporal primero  

1. Calcula y grafica ACF y PACF
2. Identifica patrones: estacionalidad, tendencia, estacionariedad
3. Usa las reglas de decisión para seleccionar el modelo base
4. Valida los residuos del modelo ajustado (deben parecer ruido blanco)

### Próximos pasos:

- Aprender sobre diferenciación para hacer series estacionarias
- Ajustar modelos ARIMA usando las pistas de ACF/PACF
- Validar modelos mediante análisis de residuos
- Explorar modelos estacionales (SARIMA)

---


# EOF (End Of File)